In [1]:
from __future__ import print_function, division
import numpy as np
np.random.seed(42)
import tensorflow as tf
import pdb
import fxns

In [39]:
# data locations
raw_data_path = '../data/shuffled_examples'
outbase = '../output/shuffled_examples'
train_out_path = '%s/texttrain' % outbase
valid_out_path = '%s/textvalidate' % outbase
test_out_path = '%s/texttest' % outbase
motif_path = '%s/../motifs' % raw_data_path

# training hyperparameters
batch_size = 50
num_batches_in_train = int(434786 / batch_size)
num_epochs = 2
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

# retrieving models
global_step_to_load = None

In [48]:
tf.reset_default_graph()

# get training data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('inputs are of size', info['seq_len'])

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)
print('outputs are of size', info['label_len'])

reload(fxns)
print(test_out_path)
(seqt, labelt), info = fxns.get_seq_and_label(test_out_path, num_epochs=None)
seqt_batch, labelt_batch = tf.train.shuffle_batch([seqt, labelt],
    batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

inputs are of size 4000
outputs are of size 24
../output/shuffled_examples/texttest


In [49]:
# # get model -- logistic
# import logreg_model
# modelname='logreg'
# X, Y, loss, logits = logreg_model.get_logreg_model(info['seq_len'], info['label_len'])

# get model -- generic convnet
import motif_model; reload(motif_model)
modelname='dumbmotifconv'
#conv_infos = [(7,(1,20),(2,2),4),(5,(1,20),(2,2),7)]#, (8,(1,20),(2,2),5)]
conv_infos = [(14,(1,21),(2,2),2,4),(14,(1,8),(2,2),2,14),(7,(1,5),(2,2),2,14)]#, (8,(1,20),(2,2),5)]
X, Y, loss, logits = motif_model.get_motif_model(info['seq_len'], info['label_len'], conv_infos, 'dumb')

21
[[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.  1.  1.  1.  1

In [50]:
# define optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate,
                                                 name='SGD-Optimizer')
    update = optimizer.minimize(loss)

# define other summaries we want
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram-loss', loss)
    summary_op = tf.summary.merge_all()

In [51]:
if tf.gfile.Exists('log/' + modelname):
   tf.gfile.DeleteRecursively('log/' + modelname) 

# train

print_every = 25
save_every = 250

with tf.Session() as sess:
    
    saver = tf.train.Saver()
    
    if global_step_to_load is None:
        global_step = 0
    else:
        saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load))
        global_step = global_step_to_load
    
#    print(layer)
    
    writer = tf.summary.FileWriter('log/' + modelname + '/train', sess.graph)
    writerv = tf.summary.FileWriter('log/' + modelname + '/valid')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    if global_step_to_load is None:
        sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(num_batches_in_train):
            batch_X, batch_Y = sess.run([seq_batch, label_batch])
            batch_Xv, batch_Yv = sess.run([seqv_batch, labelv_batch])
            _, _loss = sess.run([update, loss],
                            feed_dict={X: batch_X, Y: batch_Y})
            total_loss += _loss #/ num_batches_in_train
            
            summary = sess.run(summary_op, feed_dict={X: batch_X,
                                                    Y: batch_Y})
            summaryv = sess.run(summary_op, feed_dict={X: batch_Xv,
                                                    Y: batch_Yv})
#            writer.add_summary(summary, global_step=epoch*num_batches_in_train + i)
#            writerv.add_summary(summaryv, global_step=epoch*num_batches_in_train + i)
            writer.add_summary(summary, global_step=global_step)
            writerv.add_summary(summaryv, global_step=global_step)
            
            global_step += 1
            
            if global_step % save_every == 0:
                print('saving')
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
                saver.save(sess, 'log/%s' % modelname, global_step=global_step)
                
            if global_step % print_every == 0:
                print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))
        
        print('saving')
        print('Epoch:',epoch,'global_step',global_step,'i',i,'Avg loss in epoch(incomplete):',total_loss / (i+1))

        saver.save(sess, 'log/%s' % modelname, global_step=global_step)
    
    writer.close()
    coord.request_stop()
    coord.join(threads)

Epoch: 0 global_step 25 i 24 Avg loss in epoch(incomplete): 0.457716552615
Epoch: 0 global_step 50 i 49 Avg loss in epoch(incomplete): 0.313829144835
Epoch: 0 global_step 75 i 74 Avg loss in epoch(incomplete): 0.264358419379
Epoch: 0 global_step 100 i 99 Avg loss in epoch(incomplete): 0.239746309966
Epoch: 0 global_step 125 i 124 Avg loss in epoch(incomplete): 0.224768244267
Epoch: 0 global_step 150 i 149 Avg loss in epoch(incomplete): 0.215065217614
Epoch: 0 global_step 175 i 174 Avg loss in epoch(incomplete): 0.208021663087
Epoch: 0 global_step 200 i 199 Avg loss in epoch(incomplete): 0.202539940476
Epoch: 0 global_step 225 i 224 Avg loss in epoch(incomplete): 0.198411289056
saving
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.194952221692
Epoch: 0 global_step 250 i 249 Avg loss in epoch(incomplete): 0.194952221692
Epoch: 0 global_step 275 i 274 Avg loss in epoch(incomplete): 0.192288268967
Epoch: 0 global_step 300 i 299 Avg loss in epoch(incomplete): 0.190186198155

Epoch: 0 global_step 2425 i 2424 Avg loss in epoch(incomplete): 0.168261210531
Epoch: 0 global_step 2450 i 2449 Avg loss in epoch(incomplete): 0.168235201708
Epoch: 0 global_step 2475 i 2474 Avg loss in epoch(incomplete): 0.168192652921
saving
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.168161653554
Epoch: 0 global_step 2500 i 2499 Avg loss in epoch(incomplete): 0.168161653554
Epoch: 0 global_step 2525 i 2524 Avg loss in epoch(incomplete): 0.168133609136
Epoch: 0 global_step 2550 i 2549 Avg loss in epoch(incomplete): 0.168099024693
Epoch: 0 global_step 2575 i 2574 Avg loss in epoch(incomplete): 0.168067243632
Epoch: 0 global_step 2600 i 2599 Avg loss in epoch(incomplete): 0.168041149653
Epoch: 0 global_step 2625 i 2624 Avg loss in epoch(incomplete): 0.167999432002
Epoch: 0 global_step 2650 i 2649 Avg loss in epoch(incomplete): 0.167972691188
Epoch: 0 global_step 2675 i 2674 Avg loss in epoch(incomplete): 0.167937030809
Epoch: 0 global_step 2700 i 2699 Avg loss in 

Epoch: 0 global_step 4775 i 4774 Avg loss in epoch(incomplete): 0.166150248893
Epoch: 0 global_step 4800 i 4799 Avg loss in epoch(incomplete): 0.166137077386
Epoch: 0 global_step 4825 i 4824 Avg loss in epoch(incomplete): 0.16612235652
Epoch: 0 global_step 4850 i 4849 Avg loss in epoch(incomplete): 0.166109750102
Epoch: 0 global_step 4875 i 4874 Avg loss in epoch(incomplete): 0.166094979002
Epoch: 0 global_step 4900 i 4899 Avg loss in epoch(incomplete): 0.166078040125
Epoch: 0 global_step 4925 i 4924 Avg loss in epoch(incomplete): 0.166060635463
Epoch: 0 global_step 4950 i 4949 Avg loss in epoch(incomplete): 0.166043746625
Epoch: 0 global_step 4975 i 4974 Avg loss in epoch(incomplete): 0.166022823504
saving
Epoch: 0 global_step 5000 i 4999 Avg loss in epoch(incomplete): 0.16600438647
Epoch: 0 global_step 5000 i 4999 Avg loss in epoch(incomplete): 0.16600438647
Epoch: 0 global_step 5025 i 5024 Avg loss in epoch(incomplete): 0.165980784312
Epoch: 0 global_step 5050 i 5049 Avg loss in epo

Epoch: 0 global_step 7150 i 7149 Avg loss in epoch(incomplete): 0.16464297037
Epoch: 0 global_step 7175 i 7174 Avg loss in epoch(incomplete): 0.164630347872
Epoch: 0 global_step 7200 i 7199 Avg loss in epoch(incomplete): 0.164618962889
Epoch: 0 global_step 7225 i 7224 Avg loss in epoch(incomplete): 0.164607437498
saving
Epoch: 0 global_step 7250 i 7249 Avg loss in epoch(incomplete): 0.164593803003
Epoch: 0 global_step 7250 i 7249 Avg loss in epoch(incomplete): 0.164593803003
Epoch: 0 global_step 7275 i 7274 Avg loss in epoch(incomplete): 0.164581495216
Epoch: 0 global_step 7300 i 7299 Avg loss in epoch(incomplete): 0.164568590254
Epoch: 0 global_step 7325 i 7324 Avg loss in epoch(incomplete): 0.16455606477
Epoch: 0 global_step 7350 i 7349 Avg loss in epoch(incomplete): 0.164538365653
Epoch: 0 global_step 7375 i 7374 Avg loss in epoch(incomplete): 0.164523692169
Epoch: 0 global_step 7400 i 7399 Avg loss in epoch(incomplete): 0.164513827156
Epoch: 0 global_step 7425 i 7424 Avg loss in ep

saving
Epoch: 1 global_step 9500 i 804 Avg loss in epoch(incomplete): 0.160008446641
Epoch: 1 global_step 9500 i 804 Avg loss in epoch(incomplete): 0.160008446641
Epoch: 1 global_step 9525 i 829 Avg loss in epoch(incomplete): 0.160012243347
Epoch: 1 global_step 9550 i 854 Avg loss in epoch(incomplete): 0.160013900462
Epoch: 1 global_step 9575 i 879 Avg loss in epoch(incomplete): 0.159993293407
Epoch: 1 global_step 9600 i 904 Avg loss in epoch(incomplete): 0.160013082969
Epoch: 1 global_step 9625 i 929 Avg loss in epoch(incomplete): 0.159995003846
Epoch: 1 global_step 9650 i 954 Avg loss in epoch(incomplete): 0.159997042031
Epoch: 1 global_step 9675 i 979 Avg loss in epoch(incomplete): 0.159999943585
Epoch: 1 global_step 9700 i 1004 Avg loss in epoch(incomplete): 0.159991382337
Epoch: 1 global_step 9725 i 1029 Avg loss in epoch(incomplete): 0.159999919733
saving
Epoch: 1 global_step 9750 i 1054 Avg loss in epoch(incomplete): 0.159985014147
Epoch: 1 global_step 9750 i 1054 Avg loss in ep

Epoch: 1 global_step 11825 i 3129 Avg loss in epoch(incomplete): 0.159730889427
Epoch: 1 global_step 11850 i 3154 Avg loss in epoch(incomplete): 0.159720011799
Epoch: 1 global_step 11875 i 3179 Avg loss in epoch(incomplete): 0.159722820496
Epoch: 1 global_step 11900 i 3204 Avg loss in epoch(incomplete): 0.159723386224
Epoch: 1 global_step 11925 i 3229 Avg loss in epoch(incomplete): 0.159723421839
Epoch: 1 global_step 11950 i 3254 Avg loss in epoch(incomplete): 0.159716162589
Epoch: 1 global_step 11975 i 3279 Avg loss in epoch(incomplete): 0.159717578673
saving
Epoch: 1 global_step 12000 i 3304 Avg loss in epoch(incomplete): 0.159708403995
Epoch: 1 global_step 12000 i 3304 Avg loss in epoch(incomplete): 0.159708403995
Epoch: 1 global_step 12025 i 3329 Avg loss in epoch(incomplete): 0.15970824044
Epoch: 1 global_step 12050 i 3354 Avg loss in epoch(incomplete): 0.159695692674
Epoch: 1 global_step 12075 i 3379 Avg loss in epoch(incomplete): 0.159683679855
Epoch: 1 global_step 12100 i 3404 

Epoch: 1 global_step 14150 i 5454 Avg loss in epoch(incomplete): 0.159394189745
Epoch: 1 global_step 14175 i 5479 Avg loss in epoch(incomplete): 0.159388879659
Epoch: 1 global_step 14200 i 5504 Avg loss in epoch(incomplete): 0.159379664421
Epoch: 1 global_step 14225 i 5529 Avg loss in epoch(incomplete): 0.15937803936
saving
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.159373467536
Epoch: 1 global_step 14250 i 5554 Avg loss in epoch(incomplete): 0.159373467536
Epoch: 1 global_step 14275 i 5579 Avg loss in epoch(incomplete): 0.159370219045
Epoch: 1 global_step 14300 i 5604 Avg loss in epoch(incomplete): 0.15936077038
Epoch: 1 global_step 14325 i 5629 Avg loss in epoch(incomplete): 0.159362253626
Epoch: 1 global_step 14350 i 5654 Avg loss in epoch(incomplete): 0.159356063011
Epoch: 1 global_step 14375 i 5679 Avg loss in epoch(incomplete): 0.159356175447
Epoch: 1 global_step 14400 i 5704 Avg loss in epoch(incomplete): 0.159354413444
Epoch: 1 global_step 14425 i 5729 A

Epoch: 1 global_step 16475 i 7779 Avg loss in epoch(incomplete): 0.159109641901
saving
Epoch: 1 global_step 16500 i 7804 Avg loss in epoch(incomplete): 0.159102954405
Epoch: 1 global_step 16500 i 7804 Avg loss in epoch(incomplete): 0.159102954405
Epoch: 1 global_step 16525 i 7829 Avg loss in epoch(incomplete): 0.159096842995
Epoch: 1 global_step 16550 i 7854 Avg loss in epoch(incomplete): 0.15909424664
Epoch: 1 global_step 16575 i 7879 Avg loss in epoch(incomplete): 0.15909086998
Epoch: 1 global_step 16600 i 7904 Avg loss in epoch(incomplete): 0.159087350205
Epoch: 1 global_step 16625 i 7929 Avg loss in epoch(incomplete): 0.159078573004
Epoch: 1 global_step 16650 i 7954 Avg loss in epoch(incomplete): 0.159071750211
Epoch: 1 global_step 16675 i 7979 Avg loss in epoch(incomplete): 0.159071607242
Epoch: 1 global_step 16700 i 8004 Avg loss in epoch(incomplete): 0.159071731484
Epoch: 1 global_step 16725 i 8029 Avg loss in epoch(incomplete): 0.159064410354
saving
Epoch: 1 global_step 16750 i

In [54]:
#
# get test data

global_step_to_load_test = 17390


scores = []
labels = []

with tf.Session() as sess:
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(max_to_keep=None)
    saver.restore(sess, 'log/%s-%d' % (modelname, global_step_to_load_test))
    
    idx = 0
    while idx < 45458 / batch_size:
        batch_Xt, batch_Yt = sess.run([seqt_batch, labelt_batch])
        batch_logits = sess.run(logits, feed_dict={X: batch_Xt, Y: batch_Yt})
        scores.append(batch_logits)
        labels.append(batch_Yt)
        #print('asdf')
        idx += 1
        if idx % 100 == 0:
            print(idx)

scores_arr = np.concatenate(scores, axis=0)
labels_arr = np.concatenate(labels, axis=0)

INFO:tensorflow:Restoring parameters from log/dumbmotifconv-17390
100
200
300
400
500
600
700
800
900
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch/random_shuffle_queue, Cast_1, Cast)]]


In [55]:
accuracies = np.mean((scores_arr > 0).astype(int) == labels_arr.astype(int), axis=0)

print('accuracies:')
print(accuracies)

import sklearn.metrics
print('aucs:')
aucs = [sklearn.metrics.roc_auc_score(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(aucs)

import sklearn.metrics
def prec_recall(ys_true, ys_hat):
    ys, xs, thresholds = sklearn.metrics.precision_recall_curve(ys_true, ys_hat)
#     print(zip(xs,ys))
    return sklearn.metrics.auc(xs, ys)

print('auprcs:')
auprcs = [prec_recall(labels_arr[:,i],scores_arr[:,i]) for i in xrange(scores_arr.shape[1])]
print(auprcs)

props = np.mean(labels_arr, axis=0)

import json
ids = sorted(json.loads(info['tf_to_pos'].replace("'", '"')).keys())

from IPython.display import display_pretty, display_html
import pandas as pd
results = pd.DataFrame({'props':props, 'acc':accuracies, 'auprcs':auprcs, 'aucs':aucs}, index=ids).sort_values(by='aucs')
display_html(results.to_html(), raw=True)
results.to_csv('stats.' + modelname + '.tsv', index_label='id', sep='\t')

accuracies:
[ 0.92887912  0.9883956   0.99098901  0.99452747  0.97518681  0.97395604
  0.97516484  0.90767033  0.98136264  0.97830769  0.94145055  0.96775824
  0.9496044   0.96784615  0.90465934  0.94035165  0.94389011  0.98320879
  0.97936264  0.92076923  0.98982418  0.91635165  0.97138462  0.92907692]
aucs:
[0.57970390499110069, 0.84897469452373064, 0.67715209148099464, 0.51548830845247917, 0.69329574318195009, 0.69803365910810178, 0.50344500489647592, 0.66553373033632102, 0.72832572945664509, 0.69862531877405853, 0.60362325919625848, 0.71736309777334228, 0.53303165843762035, 0.67506511449523499, 0.7214642444771171, 0.720057221418407, 0.69399678454823266, 0.65305779947419274, 0.69433161932193843, 0.70509977473238616, 0.61081291883309197, 0.62713642089360055, 0.83174863894488482, 0.56698404825376525]
auprcs:
[0.090328430235127835, 0.045262361628946882, 0.01505762992213229, 0.0055397362376681065, 0.045095415626547827, 0.05001025825861543, 0.024803246126147472, 0.13505095708656828, 0.03

,acc,aucs,auprcs,props
HaibGm12891Pou2f2Pcr1x,0.975165,0.503445,0.024803,0.024835
HaibGm12891Pax5c20V0416101,0.994527,0.515488,0.005540,0.005473
SydhGm12891NfkbTnfaIggrab,0.949604,0.533032,0.054163,0.050396
UwHuvecCtcf,0.929077,0.566984,0.086904,0.070923
BroadHuvecCtcf,0.928879,0.579704,0.090328,0.071121
HaibHuvecPol24h8V0416101,0.941451,0.603623,0.085816,0.058549
UtaHuvecCmyc,0.989824,0.610813,0.014279,0.010176
UtaHuvecCtcf,0.916352,0.627136,0.127308,0.083648
SydhHuvecMax,0.983209,0.653058,0.026038,0.016791
HaibGm12891Pu1Pcr1x,0.907670,0.665534,0.135051,0.092330
